# Segmenting & Clustering Neighborhoods in Toronto

In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
document = lh.fromstring(page.content)
t_elements = document.xpath('//tr')

In [5]:
[len(T) for T in t_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [7]:
t_elements = document.xpath('//tr')
col=[]
z=0
for t in t_elements[0]:
    z+=1
    name=t.text_content()
    print ('%d:"%s"'%(z,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


In [14]:
for j in range(1,len(t_elements)):
    T=t_elements[j] 
    if len(T)!=3:
        break
    z=0
    for t in T.iterchildren():
        data=t.text_content() 
        if z>0:
            try:
                data=int(data)
            except:
                pass
        col[z][1].append(data)
        z+=1

In [205]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code\n,Borough\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [206]:
df.columns = ['Postcode','Borough', 'Neighbourhood']
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [207]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,,Canadian postal codes,","
1,M1A,Not assigned,"Not assigned ,Not assigned"
2,M1B,Scarborough,"Malvern, Rouge ,Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek ,Rouge ..."
4,M1E,Scarborough,"Guildwood, Morningside, West Hill ,Guildwood, ..."
5,M1G,Scarborough,"Woburn ,Woburn"
6,M1H,Scarborough,"Cedarbrae ,Cedarbrae"
7,M1J,Scarborough,"Scarborough Village ,Scarborough Village"
8,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park ,K..."
9,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge ,Golden Mile, ..."


In [231]:
df2 = df[~df.Borough.str.contains("Not assigned")]
df2 = df2.reset_index(drop=True)
df2.head(100)

,Postcode,Borough,Neighbourhood
0,,Canadian postal codes,","
1,M1B,Scarborough,"Malvern, Rouge ,Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek ,Rouge ..."
3,M1E,Scarborough,"Guildwood, Morningside, West Hill ,Guildwood, ..."
4,M1G,Scarborough,"Woburn ,Woburn"
...,...,...,...
95,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov..."
96,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe..."
97,M9L,North York,"Humber Summit ,Humber Summit"
98,M9M,North York,"Humberlea, Emery ,Humberlea, Emery"


In [232]:
df2['Neighbourhood'] = df2['Neighbourhood'].str.strip()

In [233]:
df2.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df2['Borough']

In [235]:
df2.shape

(104, 3)